In [26]:
import numpy as np
import tensorflow as tf
import os
import config
import tensorflow_hub as hub

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

print(f"Weather classes for classification:\n{config.CLASS_NAMES}")

Weather classes for classification:
['cloudy', 'dew', 'fogsmog', 'frost', 'glaze', 'hail', 'lightning', 'rain', 'rainbow', 'rime', 'sandstorm', 'shine', 'snow', 'sunrise']


In [27]:
train_data = tf.keras.utils.image_dataset_from_directory(
  config.DATA_DIR,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(config.IMG_HEIGHT, config.IMG_WIDTH),
  batch_size=config.BATCH_SIZE)

val_data = tf.keras.utils.image_dataset_from_directory(
  config.DATA_DIR,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(config.IMG_HEIGHT, config.IMG_WIDTH),
  batch_size=config.BATCH_SIZE)

test_size = int(len(val_data) / 2)
test_data = val_data.take(test_size)
val_data = val_data.skip(test_size)

Found 7987 files belonging to 14 classes.
Using 6390 files for training.
Found 7987 files belonging to 14 classes.
Using 1597 files for validation.


In [28]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
val_data = val_data.cache().prefetch(buffer_size=AUTOTUNE)
test_data = test_data.cache().prefetch(buffer_size=AUTOTUNE)

In [35]:
mobilenet_v2 = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
inception_v3 = "https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4"

feature_extractor_model = mobilenet_v2

In [42]:
feature_extractor_layer = hub.KerasLayer(
    feature_extractor_model,
    input_shape=(180, 180, 3),
    trainable=False)

In [43]:
feature_batch = feature_extractor_layer(train_data.take(1))
print(feature_batch.shape)

ValueError: Exception encountered when calling layer "keras_layer_14" "                 f"(type KerasLayer).

Could not find matching concrete function to call loaded from the SavedModel. Got:
  Positional arguments (4 total):
    * <_VariantDataset element_spec=(TensorSpec(shape=(None, 180, 180, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>
    * False
    * False
    * 0.99
  Keyword arguments: {}

 Expected these arguments to match one of the following 4 option(s):

Option 1:
  Positional arguments (4 total):
    * TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='inputs')
    * True
    * False
    * TensorSpec(shape=(), dtype=tf.float32, name='batch_norm_momentum')
  Keyword arguments: {}

Option 2:
  Positional arguments (4 total):
    * TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='inputs')
    * True
    * True
    * TensorSpec(shape=(), dtype=tf.float32, name='batch_norm_momentum')
  Keyword arguments: {}



In [37]:
model = tf.keras.Sequential([
  feature_extractor_layer,
  tf.keras.layers.Dense(config.NUM_CLASSES)
])

model.summary()

ValueError: Exception encountered when calling layer "keras_layer_11" (type KerasLayer).

in user code:

    File "c:\Users\irman\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow_hub\keras_layer.py", line 237, in call  *
        result = smart_cond.smart_cond(training,

    ValueError: Could not find matching concrete function to call loaded from the SavedModel. Got:
      Positional arguments (4 total):
        * <tf.Tensor 'inputs:0' shape=(None, 180, 180, 3) dtype=float32>
        * False
        * False
        * 0.99
      Keyword arguments: {}
    
     Expected these arguments to match one of the following 4 option(s):
    
    Option 1:
      Positional arguments (4 total):
        * TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='inputs')
        * True
        * False
        * TensorSpec(shape=(), dtype=tf.float32, name='batch_norm_momentum')
      Keyword arguments: {}
    
    Option 2:
      Positional arguments (4 total):
        * Tensor

In [11]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

history = model.fit(train_data,
                    validation_data=val_data,
                    epochs=config.EPOCHS,
                    batch_size=config.BATCH_SIZE)

Epoch 1/15


ValueError: in user code:

    File "c:\Users\irman\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\irman\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\irman\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\irman\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\irman\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\irman\AppD